In [1]:
from allennlp.predictors.predictor import Predictor
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)

error loading _jsonnet (this is expected on Windows), treating C:\Users\srina\AppData\Local\Temp\tmpg4ralbf8\config.json as plain json
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import itertools
import re
import nltk
import string
import pandas as pd
import numpy as np
import rouge
ROUGE = rouge.Rouge(metrics=['rouge-n'],
                           max_n=1)
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from nltk.corpus import stopwords
from collections import OrderedDict

In [3]:
df = pd.read_csv('datasets/wikihowAll.csv')

In [4]:
df = df.drop(['title'], axis = 1)
df = df.head(100)
df

,headline,text
0,"\nKeep related supplies in the same area.,\nMa...","If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...","As you start planning for a project or work, ..."
...,...,...
95,"\nWatch out for a shifting saddle.,\nNotice if...",If you feel that you are having to lean to on...
96,"\nBuild a trustworthy bond with your piggy.,\n...",Having a stable relationship with an animal m...
97,"\nLet your guinea pig out of its cage.,\nMake ...",";\n,,,,, Make sure every time you do this you ..."
98,\nCondition your guinea pig to the clicker or ...,"To do this, put your guinea pig in a safe are..."


In [5]:
df['headline'] = df['headline'].apply(lambda x: x.replace(',\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace(',\n', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n,', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n,', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n\n', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n\n\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n\n\n', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n', ' '))

In [6]:
df

,headline,text
0,Keep related supplies in the same area. Make ...,"If you're a photographer, keep all the necess..."
1,Create a sketch in the NeoPopRealist manner o...,See the image for how this drawing develops s...
2,Get a bachelor’s degree. Enroll in a studio-b...,It is possible to become a VFX artist without...
3,Start with some experience or interest in art...,The best art investors do their research on t...
4,"Keep your reference materials, sketches, arti...","As you start planning for a project or work, ..."
...,...,...
95,Watch out for a shifting saddle. Notice if yo...,If you feel that you are having to lean to on...
96,Build a trustworthy bond with your piggy. Res...,Having a stable relationship with an animal m...
97,Let your guinea pig out of its cage. Make sur...,"; ,,,, Make sure every time you do this you sa..."
98,Condition your guinea pig to the clicker or v...,"To do this, put your guinea pig in a safe are..."


In [7]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def sentence_segment(self, doc, lower):
        sentences = []
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sents = re.split(pattern, doc)
        sents = [sentence.split() for sentence in sents if any(char.isalpha() for char in sentence)]
        
        for sent in sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if lower is True:
                    selected_words.append(token.lower())
                else:
                    selected_words.append(token)
            sentences.append(selected_words)
        return sentences
        
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                window_size=4, lower=False):
        """Main function to analyze text"""
        
        # Filter sentences
        sentences = self.sentence_segment(text, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [8]:
def summarize_text(dataframe):
    # Filling in Nan values
    dataframe['text'] = dataframe['text'].fillna('This abstract does not exist')

    # Generating best summary for each abstract
    summaries = []
    scores = []
    resolved = []
    for i in range(len(dataframe)):
        abstract = dataframe.loc[i, 'text']
        
        # Allen Anaphora Resolution
        prediction = predictor.predict(document=abstract)  # get prediction
        abstract = predictor.coref_resolved(abstract)  # resolved text
        resolved.append(abstract)
        
        headline = dataframe.loc[i, 'headline']
        
        # TextRank
        textRank = TextRank4Keyword()
        textRank.analyze(abstract)
        text_rank_scores = textRank.node_weight

        # Sentence Tokenization
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sentences = re.split(pattern, abstract)
        sentences = [sentence for sentence in sentences if any(char.isalpha() for char in sentence)]
        headline_sentences = re.split(pattern, headline)
        headline_sentences = [sentence for sentence in headline_sentences if any(char.isalpha() for char in sentence)]
        n_sentences = len(headline_sentences)

        # Sentence scoring based on textrank values
        scored_sentences = []
        for index, sentence in enumerate(sentences):
            words = sentence.split(' ')
            score = sum(text_rank_scores[word] if word in text_rank_scores else 0.0 for word in words)
            scored_sentences.append((sentence, score, index))
        scored_sentences.sort(key=lambda x: x[1], reverse=True)

        # Top n sentences are taken as summary
        top_sentences = []
        for sentence, score, index in scored_sentences[:n_sentences]:
            top_sentences.append((sentence, index))
        top_sentences.sort(key=lambda x: x[1])
        top_sentences = [sentence for sentence, index in top_sentences]
        summary = ' '.join(top_sentences)
        summaries.append(summary)
        
        # Rouge-1 score calculation
        score = ROUGE.get_scores(summary, headline)
        p = score["rouge-1"]["p"]
        r = score["rouge-1"]["r"]
        f1 = score["rouge-1"]["f"]
        scores.append([p, r, f1])

    dataframe['resolved text'] = resolved
    dataframe['summary'] = summaries
    dataframe['rouge-1 score'] = scores
    
    return dataframe

In [9]:
df = summarize_text(df)

C:\Users\srina\OneDrive\Desktop\LinuxShared\NLP\Text Summarization\projectsum\sumenv\lib\site-packages\allennlp\modules\token_embedders\pretrained_transformer_embedder.py:385: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  num_effective_segments = (seq_lengths + self._max_length - 1) // self._max_length


In [10]:
df

,headline,text,resolved text,summary,rouge-1 score
0,Keep related supplies in the same area. Make ...,"If you're a photographer, keep all the necess...","If you're a photographer, keep all the necess...","Infrequent or secondary supplies area, tucked ...","[0.26495726495726496, 0.36470588235294116, 0.3..."
1,Create a sketch in the NeoPopRealist manner o...,See the image for how this drawing develops s...,See the image for how this drawing develops s...,"However, there is an important detail: the fol...","[0.375, 0.3684210526315789, 0.3716814159292035]"
2,Get a bachelor’s degree. Enroll in a studio-b...,It is possible to become a VFX artist without...,It is possible to become a VFX artist without...,Choose a college with a reputation for strengt...,"[0.16216216216216217, 0.46153846153846156, 0.24]"
3,Start with some experience or interest in art...,The best art investors do their research on t...,The best art investors do The best art invest...,The best art investors do The best art invest...,"[0.3884297520661157, 0.415929203539823, 0.4017..."
4,"Keep your reference materials, sketches, arti...","As you start planning for a project or work, ...","As you start planning for a project or work, ...","While everyone has a strategy, there is nothin...","[0.20689655172413793, 0.4, 0.2727272727272727]"
...,...,...,...,...,...
95,Watch out for a shifting saddle. Notice if yo...,If you feel that you are having to lean to on...,If you feel that you are having to lean to on...,If the pommel (the horn that you hold with one...,"[0.10569105691056911, 0.5, 0.174496644295302]"
96,Build a trustworthy bond with your piggy. Res...,Having a stable relationship with an animal m...,Having a stable relationship with an animal m...,If you choose a weak rope - aka a weak relatio...,"[0.140625, 0.5454545454545454, 0.2236024844720..."
97,Let your guinea pig out of its cage. Make sur...,"; ,,,, Make sure every time you do this you sa...","; ,,,, Make sure every time you do this you sa...","; ,,,, Make sure every time you do this you sa...","[0.6923076923076923, 0.08653846153846154, 0.15..."
98,Condition your guinea pig to the clicker or v...,"To do this, put your guinea pig in a safe are...","To do this, put your guinea pig in a safe are...",To see if your guinea pig is properly conditio...,"[0.1984126984126984, 0.6756756756756757, 0.306..."


In [11]:
print(df['text'][96])

 Having a stable relationship with an animal makes them so much easier to train. If your guinea pig doesn't trust you, or thinks of you as a scary giant that comes to try and communicate all of the time, it will be more difficult to teach. The trust of an animal is a very fragile thing.  Think of it as if you are a mountain climber, trying to climb the mountain of training. The peak is success, and the rope that you depend on to get you to the top is the relationship with the animal you wish to train. Would you rather have a weak rope - aka a weak relationship - or a stronger, more dependable one? If you choose the weak rope, and it snaps, you'll come tumbling back down to where you started. The strong rope has a better chance of holding up, though, be aware that if you are constantly tampering with the rope, even the strongest has a chance of breaking. If you aren't sure of your current relationship with your pet, it is best to keep working with them until they trust you enough to all

In [12]:
print(df['summary'][96])

If you choose a weak rope - aka a weak relationship, and a weak rope - aka a weak relationship snaps, you'll come tumbling back down to where you started. a stronger, more dependable one has a better chance of holding up, though, be aware that if you are constantly tampering with a stronger, more dependable one, even the strongest has a chance of breaking. If you aren't sure of your current relationship with your pet, it is best to keep working with your pet until your pet trust you enough to allow you to pick your pet up and pet your pet.  If you choose to clicker train your guinea pig, you will need a soft clicker-be sure that a soft clicker is not too loud or sharp for a guinea pig's ears-and, of course, treats.


In [13]:
print(df['headline'][96])

 Build a trustworthy bond with your piggy. Research different training methods. Choose the training method that works best for you and your guinea pig. Gather the materials that you will need for training.


In [14]:
p_sum = r_sum = f1_sum = 0
for score in df['rouge-1 score']:
    p_sum += score[0]
    r_sum += score[1]
    f1_sum += score[2]
total = len(df['rouge-1 score'])
avg = [p_sum/total, r_sum/total, f1_sum/total]
print(avg)

[0.2141972382334622, 0.46746377987242127, 0.26820622835613295]
